<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-04-08 06:56:23--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  35.7MB/s    in 7.7s    

2020-04-08 06:56:31 (32.4 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [ ]:
%tensorflow_version 1.x

import tensorflow as tf

print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [ ]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


Found 9501 images belonging to 2 classes.


In [ ]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


Found 500 images belonging to 2 classes.


In [ ]:
model = Sequential()

model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.layers[0].layers
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
step_per_epoch_test = len(test_generator)
num_epochs = 2

In [ ]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 197s 654ms/step - loss: 0.0474 - acc: 0.9853 - val_loss: 0.1570 - val_acc: 0.9343
Epoch 2/2
301/301 [==============================] - 190s 632ms/step - loss: 0.0185 - acc: 0.9921 - val_loss: 0.1315 - val_acc: 0.9490


In [ ]:
model.save('classifier_resnet50_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
ClassifierVgg16 = keras.models.load_model('classifier_vgg16_model.h5')
ClassifierResnet50 = keras.models.load_model('classifier_resnet50_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [ ]:
print(ClassifierVgg16.metrics_names)
ClassifierVgg16.evaluate_generator(test_generator, 
                   steps=step_per_epoch_test, callbacks=None, 
                   max_queue_size=10, workers=1, 
                   use_multiprocessing=True, verbose=1)

['loss', 'acc']
5/5 [==============================] - 4s 707ms/step


[0.013483520597219467, 0.9960000038146972]

In [ ]:
print(ClassifierResnet50.metrics_names)
ClassifierResnet50.evaluate_generator(test_generator, 
                   steps=step_per_epoch_test, callbacks=None, 
                   max_queue_size=10, workers=1, 
                   use_multiprocessing=True, verbose=1)

['loss', 'acc']
5/5 [==============================] - 7s 1s/step


[0.10773656293749809, 0.9600000023841858]

['loss', 'acc']

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [ ]:
predictionVgg16 =ClassifierVgg16.predict_generator(test_generator,steps=step_per_epoch_test, 
                  callbacks=None, max_queue_size=10, 
                  workers=1, use_multiprocessing=True, verbose=1)



5/5 [==============================] - 4s 714ms/step


0.99250776

In [ ]:
#VGG16
x=0
while x < 5:
  pred = predictionVgg16[x][0]
  if pred > 0.5:
    labelPred = 'Positive'
  else:
    labelPred = 'Negative'
  print(str(pred)+'-'+ str(labelPred))
  x=x+1

0.99250776-Positive
2.073607e-05-Negative
0.9883178-Positive
0.9995166-Positive
0.051881503-Negative


In [ ]:
predictionResnet50 =ClassifierResnet50.predict_generator(test_generator,steps=step_per_epoch_test, 
                  callbacks=None, max_queue_size=10, 
                  workers=1, use_multiprocessing=True, verbose=1)

5/5 [==============================] - 3s 655ms/step


In [ ]:
#ResNet50
x=0
while x < 5:
  pred = predictionResnet50[x][0]
  if pred > 0.5:
    labelPred = 'Positive'
  else:
    labelPred = 'Negative'
  print(str(pred)+'-'+ str(labelPred))
  x=x+1

0.99988985-Positive
0.999734-Positive
0.0009901744-Negative
0.99958605-Positive
0.489458-Negative


  By doing the evaluations and checking the accuracy of each model, we can say that VGG16 outperforms the resnet50 after doing the transfer learning process

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).